# Trabalho prático AC

## Trabalho realizado por:

Margarida Vila-chã, 20210504

Miguel Duarte Silva, 20210504

Sebastião Lessa, 202103238


## Algoritmo escolhido e breve explicação:

O algoritmo escolhido pelo nosso grupo foi o Random Forrest.

Este algoritmo é um algoritmo de aprendizagem computacional supervisionado. Ele usa uma combinação de árvores de decisão aleatórias. O processo começa com a criação de um grande número de árvores (floresta), em que cada uma das árvores criadas é treinada com uma amostra diferente. Isto permite a criação de árvores suficientemente diferentes, o que leva a que elas juntas consigam lidar com uma ampla variedade de situações, reduzindo o risco de overffitting.

Baseamos-nos nos seguintes data-sets:

- [breast-w](https://www.openml.org/search?type=run&id=7413&collections.id=99&run_task.task_id=15)
- [qsar-biodeg](https://www.openml.org/search?type=run&id=519587&collections.id=99&run_task.task_id=9957)
- [diabetes](https://www.openml.org/search?type=run&id=548&collections.id=99&run_task.task_id=37)


## Alteração escolhida
A alteração que o nosso grupo decidiu fazer no codigo fonte obtido foi a de considerar a utilização de outros tipos de critérios de divisão. O critério pré definido no nosso código fonte é o de entropia, por isso vamos testar com o gini e com o classification error.

## Import das bibliotecas necessárias:

In [88]:
from IPython.display import Audio
from collections import Counter
from time import time
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Modelo implementado, sem alteração

In [89]:
class DecisionTree:
    def __init__(self, criterion='gini', max_depth=None):
        self.criterion = criterion
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        label_counts = Counter(y)

        # Base cases: check termination conditions
        if len(label_counts) == 1:
            # All samples belong to the same class
            return {'class': label_counts.most_common(1)[0][0]}

        if depth == self.max_depth or num_samples < 2:
            # Reached maximum depth or too few samples remaining
            return {'class': label_counts.most_common(1)[0][0]}

        if num_features == 0:
            # No features remaining to split on
            return {'class': label_counts.most_common(1)[0][0]}

        # Determine the best feature and split point
        best_feature, best_split_point = self._get_best_split(X, y)

        # Create the tree node
        node = {'feature': best_feature, 'split_point': best_split_point}

        # Split the data based on the best feature and split point
        left_mask = X[:, best_feature] <= best_split_point
        right_mask = X[:, best_feature] > best_split_point

        left_X, left_y = X[left_mask], y[left_mask]
        right_X, right_y = X[right_mask], y[right_mask]

        # Recursively build the left and right subtrees
        node['left'] = self._build_tree(left_X, left_y, depth + 1)
        node['right'] = self._build_tree(right_X, right_y, depth + 1)

        return node

    def _compute_score(self, y):
        # Compute the score based on the selected criterion
        if self.criterion == 'gini':
            return self._gini_index(y)
        elif self.criterion == 'entropy':
            return self._entropy(y)
        elif self.criterion == 'log_loss':
            return self._log_loss(y)
        else:
            raise ValueError("Invalid criterion: {}".format(self.criterion))

    def predict(self, X):
        # Make predictions for the input data
        predictions = np.zeros(len(X))

        for i, sample in enumerate(X):
            predictions[i] = self._traverse_tree(sample, self.tree)

        return predictions

    def _traverse_tree(self, sample, node):
        if 'class' in node:
            # Leaf node: return the predicted class
            return node['class']

        # Internal node: recursively traverse the tree
        feature = node['feature']
        split_point = node['split_point']

        if sample[feature] <= split_point:
            return self._traverse_tree(sample, node['left'])
        else:
            return self._traverse_tree(sample, node['right'])

    @staticmethod
    def _gini_index(y):
        # Compute the Gini index for a given set of labels
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1.0 - np.sum(probabilities ** 2)
        return gini

    @staticmethod
    def _entropy(y):
        # Compute the entropy for a given set of labels
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    @staticmethod
    def _log_loss(y):
        # Compute the logarithmic loss for a given set of labels
        num_samples = len(y)
        unique_classes = np.unique(y)

        class_counts = Counter(y)
        class_probabilities = np.array([class_counts[c] / num_samples for c in unique_classes])

        return -np.sum(class_probabilities * np.log2(class_probabilities))

    def _get_best_split(self, X, y):
        best_feature = None
        best_split_point = None
        best_score = -np.inf

        for feature in range(X.shape[1]):
            unique_values = np.unique(X[:, feature])
            for value in unique_values:
                left_mask = X[:, feature] <= value
                right_mask = X[:, feature] > value
                left_labels = y[left_mask]
                right_labels = y[right_mask]

                if len(left_labels) > 0 and len(right_labels) > 0:
                    score = self._compute_score(left_labels) * len(left_labels) + \
                            self._compute_score(right_labels) * len(right_labels)
                    if score > best_score:
                        best_feature = feature
                        best_split_point = value
                        best_score = score

        return best_feature, best_split_point

    def get_best_criterion(self, X, y, criteria):
        best_score = 0.0
        best_criterion = None

        for criterion in criteria:
            self.criterion = criterion
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            self.fit(X_train, y_train)
            y_pred = self.predict(X_test)
            score = accuracy_score(y_test, y_pred)

            if score > best_score:
                best_score = score
                best_criterion = criterion

        return best_criterion

In [90]:
class Custom_RF(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        self.estimators = []
        # self.max_depth = self._get_best_max_depth(X, y)
        # self.n_estimators = self._get_best_n_estimators(X, y)

        criteria = ["gini", "entropy", "log_loss"]
        used_criterion=[]

        for _ in range(self.n_estimators):
            bootstrap_indices = resample(range(len(X)), replace=True)
            X_bootstrap = X[bootstrap_indices]
            y_bootstrap = y[bootstrap_indices]

            tree = DecisionTree(self.max_depth)
            best_criterion=tree.get_best_criterion(X_bootstrap, y_bootstrap, criteria)
            used_criterion.append(best_criterion)

            tree.criterion=best_criterion
            tree.fit(X_bootstrap, y_bootstrap)

            self.estimators.append(tree)

        print("Most used criterion in our implementation: '"+ str(Counter(used_criterion).most_common(1)[0][0])+ "'")

    def predict(self, X):
        predictions = np.zeros((len(X), len(self.estimators)))

        for i, estimator in enumerate(self.estimators):
            predictions[:, i] = estimator.predict(X)

        return np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)

    @staticmethod
    def _get_best_max_depth(X, y):
        param_grid = {'max_depth': [None, 5, 10, 15, 20]}
        rf_classifier = Custom_RF(n_estimators=1)  # Temporarily create an instance for grid search
        grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X, y)
        best_max_depth = grid_search.best_params_['max_depth']
        return best_max_depth

    def _get_best_n_estimators(self, X, y):
        param_grid = {'n_estimators': [50, 100, 150, 200, 250]}
        rf_classifier = Custom_RF(max_depth=self.max_depth)  # Temporarily create an instance for grid search
        grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X, y)
        best_n_estimators = grid_search.best_params_['n_estimators']
        return best_n_estimators

## Testes para cada dataset

In [91]:
def test_model(dataset):
    def test(model):
        X=dataset.iloc[:,1:-1].values
        y=dataset.iloc[:,-1].values

        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)

        model.fit(X_train, y_train)

        print("Accuracy: "+ str(round(accuracy_score(y_test, model.predict(X_test)),4)))


    print("Random Forest Classifier from sklearn using 'gini' criterion:")
    test(RF())
    print("\nRandom Forest Classifier from sklearn using 'entropy' criterion:")
    test(RF(criterion='entropy'))
    print("\nRandom Forest Classifier from sklearn using 'log_loss' criterion:")
    test(RF(criterion='log_loss'))
    print("\nOur model of Random Forest Classifier:")
    ti=time()
    test(Custom_RF())
    tf=time()
    t=tf-ti
    print("Time: "+str(int(t//60))+" minutes and "+str(round(t%60,2))+" seconds")

### DATASET 1 - Cancro da mama

In [92]:
df = pd.read_csv('breast.csv')
df.head()

,id,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1,5,1,1,1,2,1,3,1,1,benign
1,2,5,4,4,5,7,10,3,2,1,benign
2,3,3,1,1,1,2,2,3,1,1,benign
3,4,6,8,8,1,3,4,3,7,1,benign
4,5,4,1,1,3,2,1,3,1,1,benign


In [93]:
df['Class']=df['Class'].map({'benign': 0, 'malignant': 1})
df['Bare_Nuclei'].replace('?', np.nan, inplace=True)
df.dropna(subset=['Bare_Nuclei'], inplace=True)
df.head()

,id,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1,5,1,1,1,2,1,3,1,1,0
1,2,5,4,4,5,7,10,3,2,1,0
2,3,3,1,1,1,2,2,3,1,1,0
3,4,6,8,8,1,3,4,3,7,1,0
4,5,4,1,1,3,2,1,3,1,1,0


#### Tests

In [94]:
test_model(df)

Random Forest Classifier from sklearn using 'gini' criterion:
Accuracy: 0.961

Random Forest Classifier from sklearn using 'entropy' criterion:
Accuracy: 0.9561

Random Forest Classifier from sklearn using 'log_loss' criterion:
Accuracy: 0.9561

Our model of Random Forest Classifier:
Most used criterion in our implementation: 'gini'
Accuracy: 0.9561
Time: 0.0 minutes and 45.38 seconds


### Dataset 2 - Biodeg

In [95]:
df = pd.read_csv('biodeg.csv')
df.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V33,V34,V35,V36,V37,V38,V39,V40,V41,Class
0,1,3.919,2.6909,0,0,0,0,0,31.4,2,...,0,0,0,2.949,1.591,0,7.253,0,0,2
1,2,4.170,2.1144,0,0,0,0,0,30.8,1,...,0,0,0,3.315,1.967,0,7.257,0,0,2
2,3,3.932,3.2512,0,0,0,0,0,26.7,2,...,0,0,1,3.076,2.417,0,7.601,0,0,2
3,4,3.000,2.7098,0,0,0,0,0,20.0,0,...,0,0,1,3.046,5.000,0,6.690,0,0,2
4,5,4.236,3.3944,0,0,0,0,0,29.4,2,...,0,0,0,3.351,2.405,0,8.003,0,0,2


#### Tests

In [96]:
test_model(df)

Random Forest Classifier from sklearn using 'gini' criterion:
Accuracy: 0.8833

Random Forest Classifier from sklearn using 'entropy' criterion:
Accuracy: 0.8517

Random Forest Classifier from sklearn using 'log_loss' criterion:
Accuracy: 0.8644

Our model of Random Forest Classifier:
Most used criterion in our implementation: 'gini'
Accuracy: 0.6909
Time: 11.0 minutes and 8.02 seconds


### Dataset 3 - Diabetes

In [97]:
df = pd.read_csv('diabetes.csv')
df.head()

,id,'preg','plas','pres','skin','insu','mass','pedi','age','class'
0,1,6,148,72,35,0,33.6,0.627,50,tested_positive
1,2,1,85,66,29,0,26.6,0.351,31,tested_negative
2,3,8,183,64,0,0,23.3,0.672,32,tested_positive
3,4,1,89,66,23,94,28.1,0.167,21,tested_negative
4,5,0,137,40,35,168,43.1,2.288,33,tested_positive


In [98]:
df["'class'"]=df["'class'"].map({'tested_positive': 0, 'tested_negative': 1})
df.head()

,id,'preg','plas','pres','skin','insu','mass','pedi','age','class'
0,1,6,148,72,35,0,33.6,0.627,50,0
1,2,1,85,66,29,0,26.6,0.351,31,1
2,3,8,183,64,0,0,23.3,0.672,32,0
3,4,1,89,66,23,94,28.1,0.167,21,1
4,5,0,137,40,35,168,43.1,2.288,33,0


#### Tests

In [99]:
test_model(df)

Random Forest Classifier from sklearn using 'gini' criterion:
Accuracy: 0.7316

Random Forest Classifier from sklearn using 'entropy' criterion:
Accuracy: 0.7576

Random Forest Classifier from sklearn using 'log_loss' criterion:
Accuracy: 0.7489

Our model of Random Forest Classifier:
Most used criterion in our implementation: 'gini'
Accuracy: 0.6797
Time: 2.0 minutes and 18.22 seconds


In [102]:
Audio('sound.mp3', autoplay=True)